In [1]:
from tqdm.notebook import tqdm
from conformer.tokenizer import Tokenizer
from conformer.dataset import batch_fn, ProcessAudioData, unpack_speech_data
import grain
from pathlib import Path
from flax import nnx
import numpy as np
import jax
import jax.numpy as jnp

In [2]:
tokenizer = Tokenizer.load_tokenizer(Path('/home/penguin/data/tinyvoice/tokenizer/tokenizer.pkl'))

In [3]:
train_audio_source = grain.sources.ArrayRecordDataSource('/home/penguin/data/tinyvoice/data/data.array_record')
# test_audio_source = grain.sources.ArrayRecordDataSource('/home/penguin/Data/processed/test.array_record')

In [4]:
map_train_audio_dataset = grain.MapDataset.source(train_audio_source)
# map_test_audio_dataset = grain.MapDataset.source(test_audio_source)

In [5]:
processed_train_dataset = (
    map_train_audio_dataset
    .shuffle(seed=42)
    .map(ProcessAudioData(tokenizer))
    .batch(batch_size=48, batch_fn=batch_fn)
    .repeat(1)
)

# processed_test_dataset = (
#     map_test_audio_dataset
#     .map(ProcessAudioData(tokenizer))
#     .batch(batch_size=8, batch_fn=batch_fn)
# )

In [6]:
from conformer.model import ConformerModel
from tqdm import tqdm

In [7]:
model = ConformerModel(token_count=len(tokenizer.id_to_char))

W1220 16:44:35.024277  107510 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1220 16:44:35.026201  107260 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.


In [8]:
import optax

In [9]:
lr_schedule = optax.linear_schedule(
    init_value=1e-7, 
    end_value=5e-4, 
    transition_steps=300
)

optimizer = nnx.Optimizer(
    model,
    optax.adamw(
        learning_rate=lr_schedule,
        b1=0.9,
        b2=0.98,
        weight_decay=1e-2
    ),
    wrt=nnx.Param
)

In [10]:
@nnx.jit
def jitted_train(model, optimizer, padded_audios, padded_labels, mask, real_times, label_lengths):
    def loss_fn(model):
        logits = model(padded_audios, mask=mask, training=True)
        log_probs = jax.nn.log_softmax(logits, axis=-1)
        
        audio_time_mask = jnp.arange(log_probs.shape[1]) >= real_times[:, None]
        label_mask = jnp.arange(padded_labels.shape[1]) >= label_lengths[:, None]
        
        loss = optax.ctc_loss(log_probs, audio_time_mask, padded_labels, label_mask).mean()

        return loss
    
    loss, grads = nnx.value_and_grad(loss_fn)(model)
    optimizer.update(model=model, grads=grads)

    return loss

In [11]:
padded_audios, frames, padded_labels, label_lengths = processed_train_dataset[43]

In [12]:
def compute_mask(frames):
    # MelSpectrogram: hop_length=160, win_length=400, padded=False
    # T_mel = (T_audio - win_length) // hop_length + 1
    # Conv2dSubSampler: two layers of kernel=3, stride=2, padding='VALID'
    # T_out = (T_in - 3) // 2 + 1
    # T_final = (T_out - 3) // 2 + 1
    
    t_mel = (frames - 400) // 160 + 1
    t_conv1 = (t_mel - 3) // 2 + 1
    t_final = (t_conv1 - 3) // 2 + 1
    
    max_frames = 235008
    max_t_mel = (max_frames - 400) // 160 + 1
    max_t_conv1 = (max_t_mel - 3) // 2 + 1
    max_t_final = (max_t_conv1 - 3) // 2 + 1

    real_times = t_final
    
    # Square mask for attention
    mask = jnp.arange(max_t_final) < real_times[:, None]
    mask = jnp.expand_dims(mask, axis=1).repeat(max_t_final, axis=1)
    
    # MultiHeadAttention mask: (batch, num_heads, q_len, k_len)
    mask = jnp.expand_dims(mask, axis=1).repeat(4, axis=1)

    return mask, real_times

In [13]:
mask, real_times = compute_mask(frames)

In [14]:
z = jitted_train(model, optimizer, padded_audios, padded_labels, mask, real_times, label_lengths)

In [39]:
avg_loss = 0
for i, element in enumerate(tqdm(processed_train_dataset)):
    padded_audios, frames, padded_labels, label_lengths = element
    mask, real_times = compute_mask(frames)

    loss = jitted_train(model, optimizer, padded_audios, padded_labels, mask, real_times, label_lengths)

    avg_loss += loss    
    if (i + 1) % 20 == 0:
        print(f"avg loss: {avg_loss // 20}")
        avg_loss = 0

  1%|          | 21/2444 [00:05<10:22,  3.89it/s]

avg loss: 42.0


  2%|▏         | 40/2444 [00:10<10:59,  3.64it/s]

avg loss: 41.0


  2%|▏         | 61/2444 [00:16<09:55,  4.00it/s]

...

 99%|█████████▉| 2421/2444 [10:07<00:05,  4.21it/s]

avg loss: 30.0


100%|█████████▉| 2441/2444 [10:12<00:00,  4.41it/s]

avg loss: 32.0


100%|██████████| 2444/2444 [10:13<00:00,  3.99it/s]


In [42]:
output = model(padded_audios, mask=mask, training=False)

In [43]:
output.shape

(46, 366, 44)

In [59]:
def decode(ids: list[int]) -> str:
    last_char_id = 0
    decoded_chars = []
    for char_id in ids:
        if char_id != 0 and char_id != last_char_id:
            decoded_chars.append(char_id)
        last_char_id = char_id
    
    return decoded_chars

In [60]:
dds = decode(output[18].argmax(axis=-1).tolist())

In [61]:
tokens = tokenizer.decode(dds)

In [62]:
# tokens = tokenizer.decode(output[10].argmax(axis=-1).tolist())

for tok in tokens:
    if tok != '<BLANK>':
        print(tok, end='')

იგი იყო ყუბაძლო მეომარიდა მხედართ მთავარი

In [67]:
z = tokenizer.decode(padded_labels[18].tolist())

In [68]:
for tok in z:
    if tok != '<BLANK>':
        print(tok, end='')

იგი იყო უბადლო მეომარი და მხედართმთავარი